# 03_model_comparison_and_analysis

**Purpose:** Systematically benchmark multiple classifier × regressor hybrid combinations, pick the best pair, calibrate classifier, retrain regressor on log1p target, compute permutation importances, and save final artifacts.


In [ ]:
import os
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import joblib

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LogisticRegression, ElasticNet
from sklearn.metrics import roc_auc_score, mean_absolute_error, mean_squared_error, roc_curve
from sklearn.inspection import permutation_importance
from sklearn.calibration import CalibratedClassifierCV

ARTIFACTS_DIR = Path("final_artifacts")
PLOTS_DIR = ARTIFACTS_DIR / "plots"
PROCESSED_PATH = Path("processed/features_ready.csv")
RANDOM_STATE = 42
THRESHOLD = 50.0


In [ ]:
df = pd.read_csv(PROCESSED_PATH, index_col=0, parse_dates=True)
exclude = ['Up_next_hour', 'Up']
X = df[[c for c in df.columns if c not in exclude]].copy()
y = df['Up_next_hour'].copy()

SPLIT_DATE = pd.Timestamp("2025-02-15 23:00:00+00:00")
train_mask = X.index <= SPLIT_DATE
X_train, X_test = X.loc[train_mask], X.loc[~train_mask]
y_train, y_test = y.loc[train_mask], y.loc[~train_mask]
y_train_event = (y_train > THRESHOLD).astype(int)
y_test_event = (y_test > THRESHOLD).astype(int)
print("Loaded data. Train/Test shapes:", X_train.shape, X_test.shape)


## Candidate classifiers and regressors
We'll try:
- Classifiers: XGBoost (if available), RandomForest, LogisticRegression
- Regressors: XGBoost (if available), RandomForestReg, GradientBoosting, ElasticNet


In [ ]:
# try xgboost
try:
    import xgboost as xgb
    XGB_AVAILABLE = True
except Exception:
    XGB_AVAILABLE = False

classifiers = []
if XGB_AVAILABLE:
    classifiers.append(("XGBClassifier", lambda: xgb.XGBClassifier(n_estimators=200, learning_rate=0.05, max_depth=5, use_label_encoder=False, eval_metric='logloss', n_jobs=-1, random_state=RANDOM_STATE)))
classifiers += [
    ("RandomForest", lambda: RandomForestClassifier(n_estimators=200, max_depth=10, n_jobs=-1, random_state=RANDOM_STATE)),
    ("LogisticRegression", lambda: LogisticRegression(max_iter=1000, solver='liblinear'))
]

regressors = []
if XGB_AVAILABLE:
    regressors.append(("XGBRegressor", lambda: xgb.XGBRegressor(n_estimators=300, learning_rate=0.05, max_depth=6, n_jobs=-1, random_state=RANDOM_STATE)))
regressors += [
    ("RandomForestReg", lambda: RandomForestRegressor(n_estimators=200, max_depth=10, n_jobs=-1, random_state=RANDOM_STATE)),
    ("GradientBoosting", lambda: GradientBoostingRegressor(n_estimators=200, learning_rate=0.05, max_depth=6, random_state=RANDOM_STATE)),
    ("ElasticNet", lambda: ElasticNet(alpha=0.1, l1_ratio=0.5, max_iter=10000, random_state=RANDOM_STATE))
]


In [ ]:
clf_models = {}
for name, ctor in classifiers:
    print("Training classifier:", name)
    pipe = Pipeline([("imputer", SimpleImputer(strategy="median")), ("clf", ctor())])
    pipe.fit(X_train, y_train_event)
    y_proba = pipe.predict_proba(X_test)[:,1]
    roc = roc_auc_score(y_test_event, y_proba)
    clf_models[name] = {"pipe": pipe, "proba": y_proba, "roc_auc": roc}
    print(" ROC-AUC:", roc)


In [ ]:
reg_models = {}
event_mask = y_train > THRESHOLD
n_events = event_mask.sum()
print("Event rows in train:", n_events)
for name, ctor in regressors:
    if n_events < 10:
        print("Skipping regressor", name, "due to too few events.")
        continue
    print("Training regressor:", name)
    pipe = Pipeline([("imputer", SimpleImputer(strategy="median")), ("reg", ctor())])
    pipe.fit(X_train.loc[event_mask], y_train.loc[event_mask].values)
    train_preds = pipe.predict(X_train.loc[event_mask])
    train_mae = mean_absolute_error(y_train.loc[event_mask], train_preds)
    reg_models[name] = {"pipe": pipe, "train_mae": train_mae}
    print(" Train-event MAE:", train_mae)


In [ ]:
results = []
for clf_name, clf_info in clf_models.items():
    y_proba = clf_info["proba"]
    for reg_name, reg_info in reg_models.items():
        reg_pipe = reg_info["pipe"]
        reg_pred_all = reg_pipe.predict(X_test)
        pw_pred = y_proba * reg_pred_all
        mae_pw = mean_absolute_error(y_test, pw_pred)
        rmse_pw = mean_squared_error(y_test, pw_pred, squared=False)
        # hard50
        hard_mask = y_proba > 0.5
        hard_pred = np.zeros_like(y_test.values, dtype=float)
        if hard_mask.sum() > 0:
            hard_pred[hard_mask] = reg_pred_all[hard_mask]
        mae_hard = mean_absolute_error(y_test, hard_pred)
        # best threshold search
        best_th, best_mae = None, np.inf
        for th in np.linspace(0,1,101):
            mask_th = y_proba > th
            p = np.zeros_like(y_test.values, dtype=float)
            if mask_th.sum() > 0:
                p[mask_th] = reg_pred_all[mask_th]
            m = mean_absolute_error(y_test, p)
            if m < best_mae:
                best_mae = m
                best_th = th
        # reg metrics on true events
        true_mask = y_test > THRESHOLD
        reg_test_mae = np.nan
        if true_mask.sum() > 0:
            reg_test_mae = mean_absolute_error(y_test[true_mask], reg_pred_all[true_mask])
        results.append({
            "classifier": clf_name,
            "classifier_roc_auc": clf_info["roc_auc"],
            "regressor": reg_name,
            "reg_train_mae_on_events": reg_info["train_mae"],
            "reg_test_mae_on_events": reg_test_mae,
            "prob_weighted_mae": mae_pw,
            "prob_weighted_rmse": rmse_pw,
            "hard50_mae": mae_hard,
            "best_th": best_th,
            "best_th_mae": best_mae
        })
        print(f"{clf_name} + {reg_name}: PW MAE {mae_pw:.3f}, hard50 MAE {mae_hard:.3f}, best_th {best_th:.2f} (MAE {best_mae:.3f})")

summary_df = pd.DataFrame(results).sort_values("prob_weighted_mae").reset_index(drop=True)
summary_df.to_csv(ARTIFACTS_DIR / "hybrid_model_comparison.csv", index=False)
display(summary_df.head(10))
print("Saved hybrid_model_comparison.csv")


## Select the best performing combo and retrain final pipelines with calibration & log1p regressor
We will:
- pick the top combo (lowest prob_weighted_mae),
- calibrate classifier probabilities (isotonic),
- retrain regressor on log1p(Up) for event rows and back-transform predictions,
- save final artifacts and compute final metrics.


In [ ]:
# pick best
best_row = summary_df.iloc[0]
best_clf = best_row['classifier']
best_reg = best_row['regressor']
print("Best combo:", best_clf, "+", best_reg)

best_clf_pipe = clf_models[best_clf]['pipe']
best_reg_pipe = reg_models[best_reg]['pipe']

# calibrate classifier
raw_clf = best_clf_pipe.named_steps['clf']
cal = CalibratedClassifierCV(raw_clf, cv=5, method='isotonic')
clf_cal_pipe = Pipeline([("imputer", SimpleImputer(strategy="median")), ("cal", cal)])
clf_cal_pipe.fit(X_train, y_train_event)
y_proba_cal = clf_cal_pipe.predict_proba(X_test)[:,1]
print("Calibrated ROC-AUC:", roc_auc_score(y_test_event, y_proba_cal).round(4))

# retrain regressor on log1p
event_mask_train = y_train > THRESHOLD
if event_mask_train.sum() == 0:
    raise RuntimeError("No event rows for regressor retrain.")
# try to reuse same regressor class
RegCtor = reg_models[best_reg]['pipe'].named_steps['reg'].__class__
try:
    reg_params = reg_models[best_reg]['pipe'].named_steps['reg'].get_params()
    reg_final = RegCtor(**{k:v for k,v in reg_params.items() if k in RegCtor().get_params()})
except Exception:
    reg_final = RegCtor()
reg_pipe_log = Pipeline([("imputer", SimpleImputer(strategy="median")), ("reg", reg_final)])
y_train_log = np.log1p(y_train.loc[event_mask_train])
reg_pipe_log.fit(X_train.loc[event_mask_train], y_train_log)
joblib.dump(clf_cal_pipe, ARTIFACTS_DIR / "classifier_calibrated.joblib")
joblib.dump(reg_pipe_log, ARTIFACTS_DIR / "regressor_log1p.joblib")
print("Saved calibrated classifier and log-regressor.")


In [ ]:
# final preds
y_proba_final = clf_cal_pipe.predict_proba(X_test)[:,1]
reg_pred_log = reg_pipe_log.predict(X_test)
reg_pred = np.expm1(reg_pred_log).clip(0)
hybrid_pw = y_proba_final * reg_pred

mae_pw = mean_absolute_error(y_test, hybrid_pw)
rmse_pw = mean_squared_error(y_test, hybrid_pw, squared=False)
print("Final prob-weighted MAE:", mae_pw, "RMSE:", rmse_pw)

# save predictions
final_preds = pd.DataFrame({
    "datetime": X_test.index,
    "y_true": y_test.values,
    "p_event": y_proba_final,
    "pred_prob_weighted": hybrid_pw
}).set_index("datetime")
final_preds.to_csv(ARTIFACTS_DIR / "final_hybrid_predictions.csv")
print("Saved final_hybrid_predictions.csv")

# permutation importances (warning: can be slow)
clf_imp = permutation_importance(clf_cal_pipe, X_test, y_test_event, n_repeats=16, random_state=RANDOM_STATE, scoring='roc_auc', n_jobs=-1)
clf_imp_df = pd.DataFrame({"feature": X_test.columns, "mean_decrease_auc": clf_imp.importances_mean}).sort_values("mean_decrease_auc", ascending=False)
clf_imp_df.head(20).to_csv(ARTIFACTS_DIR / "classifier_permutation_importance.csv", index=False)

reg_imp = permutation_importance(reg_pipe_log, X_test, np.expm1(reg_pipe_log.predict(X_test)), n_repeats=16, random_state=RANDOM_STATE, scoring='neg_mean_absolute_error', n_jobs=-1)
reg_imp_df = pd.DataFrame({"feature": X_test.columns, "mean_decrease_negMAE": reg_imp.importances_mean})
reg_imp_df["mean_increase_MAE"] = -reg_imp_df["mean_decrease_negMAE"]
reg_imp_df = reg_imp_df.sort_values("mean_increase_MAE", ascending=False)
reg_imp_df.head(20).to_csv(ARTIFACTS_DIR / "regressor_permutation_importance.csv", index=False)
print("Saved permutation importances.")


## Wrap-up
- All artifacts saved to `final_artifacts/`.
- Inspect `hybrid_model_comparison.csv`, `final_hybrid_predictions.csv`, and permutation importance CSVs for reporting and README visuals.
